<H3>Install libs<H3>

In [5]:
# Cài đặt các thư viện chính cho mô hình ngôn ngữ, embedding, RAG và giao diện web
!pip install -q \
  "torch>=2.0.0" \
  "transformers>=4.40.0" \
  "accelerate>=0.30.0" \
  "huggingface-hub>=0.23.0" \
  "sentence-transformers>=2.7.0" \
  "langchain>=0.2.0" \
  "langchain-core>=0.2.0" \
  "langchain-community>=0.1.0" \
  "langchain-text-splitters>=0.2.0" \
  "chromadb>=0.5.0" \
  "langchain-chroma>=0.2.0" \
  "pypdf>=4.2.0" \
  "gradio>=5.0.0" \
  "langchain-huggingface" \
  "wget" \
  "tqdm" \
  "ipywidgets"

/media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/.venv/bin/pip: 2: exec: /media/mtl/DATA 6TB/PROJECT AI/RAG-NLP/.venv/bin/python: not found


In [6]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


<H3>Setup project + tạo cấu trúc thư mục</H3>

In [7]:
import os, sys

# Root dự án: dùng folder "rag_langchain" nằm cùng cấp notebook (như ảnh bạn)
PROJECT_ROOT = os.path.abspath("rag_langchain")

DATA_DIR = os.path.join(PROJECT_ROOT, "data_source", "generative_ai")  # bạn copy PDF vào đây
CUSTOM_DIR = os.path.join(PROJECT_ROOT, "data_source", "custom")       # tuỳ chọn
CHROMA_DIR = os.path.join(PROJECT_ROOT, "chroma_data")                 # lưu vector DB

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CUSTOM_DIR, exist_ok=True)
os.makedirs(CHROMA_DIR, exist_ok=True)

# src (tuỳ chọn, cho đúng cấu trúc tài liệu)
os.makedirs(os.path.join(PROJECT_ROOT, "src", "base"), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, "src", "rag"), exist_ok=True)

# tạo __init__.py
for p in [
    os.path.join(PROJECT_ROOT, "src", "__init__.py"),
    os.path.join(PROJECT_ROOT, "src", "base", "__init__.py"),
    os.path.join(PROJECT_ROOT, "src", "rag", "__init__.py"),
]:
    if not os.path.exists(p):
        open(p, "w", encoding="utf-8").close()

# thêm PROJECT_ROOT vào sys.path (phòng khi tách code)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("✅ Copy PDF vào:", DATA_DIR)
print("✅ (Optional) Copy PDF khác vào:", CUSTOM_DIR)
print("✅ Chroma DB lưu ở:", CHROMA_DIR)


✅ PROJECT_ROOT: /media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/rag_langchain
✅ Copy PDF vào: /media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/rag_langchain/data_source/generative_ai
✅ (Optional) Copy PDF khác vào: /media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/rag_langchain/data_source/custom
✅ Chroma DB lưu ở: /media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/rag_langchain/chroma_data


<h3>Check dữ liệu PDF đã có chưa</h3>

In [8]:
import glob

pdf_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.pdf")))
print("📄 Số PDF trong generative_ai:", len(pdf_files))
for f in pdf_files[:20]:
    print(" -", os.path.basename(f))

if len(pdf_files) == 0:
    raise ValueError(
        "❌ Chưa có PDF!\n"
        f"Hãy copy vài file .pdf vào folder:\n{DATA_DIR}\n"
        "Rồi chạy lại cell này."
    )


📄 Số PDF trong generative_ai: 8
 - 05-vbhn-bgtvt.pdf
 - 168-nd-cp.signed.pdf
 - 2023_1335 + 1336_12-VBHN-BGTVT.pdf
 - 35-2024-qh15.pdf
 - 36-2024-qh15.pdf
 - 36-2024-qh15_tiep.pdf
 - 51-bgtvt-kem.pdf
 - 73-bca.pdf


<H3>Clean text + Loader + Chunking</H3>

In [9]:
import re
import unicodedata
from typing import List
from tqdm import tqdm

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def clean_vietnamese_text(text: str) -> str:
    # Chuẩn hóa Unicode tiếng Việt
    text = unicodedata.normalize("NFC", text)

    # Loại bỏ ký tự control (giữ \n \t)
    text = "".join(
        ch for ch in text
        if (not unicodedata.category(ch).startswith("C")) or ch in "\n\t"
    )

    # Gộp khoảng trắng thừa
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n\s*\n", "\n", text)

    return text.strip()

class SimpleLoader:
    def load_pdf(self, pdf_file: str):
        docs = PyPDFLoader(pdf_file, extract_images=True).load()
        for doc in docs:
            doc.page_content = clean_vietnamese_text(doc.page_content)
            # thêm metadata để debug (file name + page)
            doc.metadata["source_file"] = os.path.basename(pdf_file)
        return docs

    def load_dir(self, dir_path: str) -> List:
        pdfs = sorted(glob.glob(os.path.join(dir_path, "*.pdf")))
        if not pdfs:
            raise ValueError(f"No PDF files found in: {dir_path}")

        all_docs = []
        for pdf in tqdm(pdfs, desc="Loading PDFs"):
            try:
                all_docs.extend(self.load_pdf(pdf))
            except Exception as e:
                print("Skip:", pdf, "|", e)
        return all_docs

class TextSplitter:
    def __init__(self, chunk_size: int = 400, chunk_overlap: int = 120):
        self.splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n", " ", ""],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

    def split(self, documents):
        return self.splitter.split_documents(documents)


<H3>Vector DB (Chroma + Embeddings) - Hỗ trợ “thêm tài liệu” (incremental)</H3>

In [10]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

class VectorDB:
    def __init__(
        self,
        documents=None,
        #embedding_model: str = "dangvantuan/vietnamese-document-embedding",
        embedding_model: str = "keepitreal/vietnamese-sbert",
        collection_name: str = "vietnamese_docs",
        persist_dir: str = None,
    ):
        self.persist_dir = persist_dir or CHROMA_DIR
        self.collection_name = collection_name

        # ✅ Bật trust_remote_code cho model có custom code
        self.embedding = HuggingFaceEmbeddings(
            model_name=embedding_model,
            model_kwargs={"trust_remote_code": True},
        )

        self.db = Chroma(
            collection_name=self.collection_name,
            embedding_function=self.embedding,
            persist_directory=self.persist_dir,
        )

        if documents and len(documents) > 0:
            self.add_documents(documents)

    def add_documents(self, documents):
        self.db.add_documents(documents)
        if hasattr(self.db, "persist"):
            try:
                self.db.persist()
            except:
                pass

    '''
    #similarity thường
    def get_retriever(self, k: int = 4):
        return self.db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    '''

    #MMR để bớt trùng đoạn + phủ ý rộng hơn:
    def get_retriever(self, k: int = 8):
        return self.db.as_retriever(
            search_type="mmr",
            search_kwargs={"k": k, "fetch_k": max(20, k*4), "lambda_mult": 0.2}
    )



<H3>LLM (Qwen) + fallback model nhỏ cho máy yếu</H3>

In [11]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain_huggingface import HuggingFacePipeline

def get_hf_llm(
    model_name: str = "Qwen/Qwen2.5-3B-Instruct",
    temperature: float = 0.2, # thấp thì đổ lan man = chính xác hơn
    max_new_tokens: int = 900,
):
    # Nếu máy yếu / không GPU -> dùng model nhỏ cho chắc
    if not torch.cuda.is_available() and model_name == "Qwen/Qwen2.5-3B-Instruct":
        model_name = "Qwen/Qwen2.5-0.5B-Instruct"
        print("⚠️ Không thấy GPU -> auto dùng model nhỏ:", model_name)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
        low_cpu_mem_usage=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    gen_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.75,
    )

    return HuggingFacePipeline(pipeline=gen_pipe)


<H3>Prompt + Parser + RAG chain (kèm hiển thị context)</H3>

In [12]:
import re
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

class FocusedAnswerParser(StrOutputParser):
    def parse(self, text: str) -> str:
        text = (text or "").strip()

        m = re.search(r"\[?\s*TRẢ\s*LỜI\s*\]?\s*[:：]\s*", text, flags=re.IGNORECASE)
        if m:
            text = text[m.end():].strip()

        text = re.sub(r"[ \t]+\n", "\n", text)
        text = re.sub(r"\n{3,}", "\n\n", text).strip()
        return text

class OfflineRAG:
    def __init__(self, llm):
        self.llm = llm
        self.prompt = PromptTemplate.from_template("""
Bạn là trợ lý AI trả lời dựa trên tài liệu pháp luật tiếng Việt.

[TÀI LIỆU]:
{context}

[CÂU HỎI]:
{question}

QUY TẮC:
1) CHỈ dùng thông tin có trong [TÀI LIỆU]. Không suy diễn, không thêm kiến thức ngoài.
2) Nếu tài liệu KHÔNG có nội dung hỏi tới: nói rõ "Không tìm thấy trong các đoạn trích hiện có" và gợi ý người dùng nạp đúng văn bản.
3) Trả lời theo đúng mẫu dưới đây. Mỗi ý phải kèm nguồn (file | page).

MẪU TRẢ LỜI (bắt buộc):
- Tóm tắt nhanh (2–4 gạch đầu dòng).
- Trích ý theo Điều/Khoản (nếu tài liệu có):
  * Điều X (nếu có) — Ý chính: ...
    - Khoản ...: ...
    - (Nguồn: file | page)
- Kết luận: 1–2 câu.

[TRẢ LỜI]:
""".strip())
        self.answer_parser = FocusedAnswerParser()

    def get_chain(self, retriever):
        def format_docs(docs):
            blocks = []
            seen = set()
            for d in docs:
                content = (d.page_content or "").strip()
                if len(content) < 40:
                    continue
                src = d.metadata.get("source_file", "unknown")
                page = d.metadata.get("page", "?")

                key = (src, page, hash(content))
                if key in seen:
                    continue
                seen.add(key)

                blocks.append(f"[{src} | page {page}]\n{content}")
            return "\n\n---\n\n".join(blocks)

        return (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | self.answer_parser
        )

    def get_context_only(self, retriever):
        def format_docs(docs):
            blocks = []
            for d in docs:
                src = d.metadata.get("source_file", "unknown")
                page = d.metadata.get("page", "?")
                blocks.append(f"[{src} | page {page}]\n{(d.page_content or '').strip()}")
            return "\n\n---\n\n".join(blocks)
        return retriever | format_docs


<H3>Build pipeline: load → chunk → vector → chain</H3>

In [13]:
# 1) Loader + Splitter
loader = SimpleLoader()
splitter = TextSplitter(chunk_size=400, chunk_overlap=120)

# 2) VectorDB (persist)
vdb = VectorDB(documents=None)  # mở DB từ persist_dir

# Nếu muốn rebuild sạch DB mỗi lần chạy notebook, uncomment block dưới:
# import shutil
# if os.path.exists(CHROMA_DIR):
#     shutil.rmtree(CHROMA_DIR, ignore_errors=True)
# vdb = VectorDB(documents=None)

# 3) LLM + RAG
llm = get_hf_llm()
rag = OfflineRAG(llm)

# 4) Nếu trong folder đã có PDF thì ingest 1 lần (để có dữ liệu)
pdfs = sorted(glob.glob(os.path.join(DATA_DIR, "*.pdf")))
if len(pdfs) > 0:
    raw_docs = loader.load_dir(DATA_DIR)
    split_docs = splitter.split(raw_docs)
    vdb.add_documents(split_docs)

retriever = vdb.get_retriever(k=10)
rag_chain = rag.get_chain(retriever)
ctx_chain = rag.get_context_only(retriever)

def answer_question(question: str) -> str:
    try:
        return rag_chain.invoke(question)
    except Exception as e:
        return f"Error: {str(e)}"

def get_context(question: str) -> str:
    try:
        return ctx_chain.invoke(question)
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Ready. PDFs in kho:", len(glob.glob(os.path.join(DATA_DIR, '*.pdf'))))


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

...

Skip: /media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/rag_langchain/data_source/generative_ai/168-nd-cp.signed.pdf | cannot reshape array of size 41654 into shape (2357,1670,newaxis)


Loading PDFs:  88%|████████▊ | 7/8 [00:05<00:00,  1.24it/s]

Skip: /media/mtl/DATA 6TB5/PROJECT AI/RAG-NLP/rag_langchain/data_source/generative_ai/51-bgtvt-kem.pdf | cannot reshape array of size 2819 into shape (93,97,newaxis)


Loading PDFs: 100%|██████████| 8/8 [00:08<00:00,  1.04s/it]


✅ Ready. PDFs in kho: 8


<h3>“ingest pipeline” cho file upload (copy → load → chunk → add → refresh chain)</h3>

In [14]:
import shutil
import glob
import os

def list_pdfs_md(data_dir: str) -> str:
    pdfs = sorted(glob.glob(os.path.join(data_dir, "*.pdf")))
    if not pdfs:
        return "*(Chưa có file PDF nào trong kho)*"
    lines = [f"- {os.path.basename(p)}" for p in pdfs]
    return "\n".join(lines)

def ingest_uploaded_pdfs(uploaded_files, loader, splitter, vdb, data_dir: str):
    """
    uploaded_files: list[gradio UploadedFile] hoặc list[path]
    - copy vào data_dir
    - load -> chunk
    - add vào Chroma
    """
    if not uploaded_files:
        return "❌ Bạn chưa chọn file PDF nào."

    saved = []
    for f in uploaded_files:
        # gr.File có thể trả object có thuộc tính .name hoặc là string path
        src_path = getattr(f, "name", None) or str(f)
        base = os.path.basename(src_path)
        dst_path = os.path.join(data_dir, base)

        # copy vào kho
        shutil.copy2(src_path, dst_path)
        saved.append(dst_path)

    # load + chunk chỉ những file mới
    new_docs = []
    for p in saved:
        try:
            new_docs.extend(loader.load_pdf(p))
        except Exception as e:
            print("Skip load:", p, e)

    if not new_docs:
        return "⚠️ Copy xong nhưng không đọc được PDF (có thể PDF scan ảnh / lỗi định dạng)."

    new_chunks = splitter.split(new_docs)
    if not new_chunks:
        return "⚠️ Đọc được nhưng chunk rỗng (PDF có thể toàn ảnh)."

    # add vào Vector DB
    vdb.add_documents(new_chunks)

    return f"✅ Đã nạp {len(saved)} file | pages loaded: {len(new_docs)} | chunks added: {len(new_chunks)}"


<H3>Gradio UI</H3>

<H4>Chống trùng ingest bằng HASH + chỉ ingest file mới<H4>

In [15]:
import os, json, hashlib, shutil, glob
from datetime import datetime

HASH_DB_PATH = os.path.join(PROJECT_ROOT, "ingested_hashes.json")

def load_hash_db():
    if os.path.exists(HASH_DB_PATH):
        try:
            with open(HASH_DB_PATH, "r", encoding="utf-8") as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_hash_db(db):
    with open(HASH_DB_PATH, "w", encoding="utf-8") as f:
        json.dump(db, f, ensure_ascii=False, indent=2)

def sha256_file(path, chunk_size=1024*1024):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

HASH_DB = load_hash_db()

def ingest_uploaded_pdfs_no_dup(uploaded_files, loader, splitter, vdb, data_dir: str):
    """
    - copy file vào kho
    - tính hash
    - nếu hash đã có => skip
    - nếu mới => load->chunk->add_documents
    """
    if not uploaded_files:
        return "❌ Bạn chưa chọn file PDF nào."

    added_files, skipped_files = [], []
    new_docs_total, new_chunks_total = 0, 0

    for f in uploaded_files:
        src_path = getattr(f, "name", None) or str(f)
        base = os.path.basename(src_path)
        dst_path = os.path.join(data_dir, base)

        # copy vào kho (nếu trùng tên thì thêm timestamp)
        if os.path.exists(dst_path):
            name, ext = os.path.splitext(base)
            dst_path = os.path.join(data_dir, f"{name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}{ext}")

        shutil.copy2(src_path, dst_path)

        file_hash = sha256_file(dst_path)

        if file_hash in HASH_DB:
            skipped_files.append(os.path.basename(dst_path))
            # file trùng hash thì xóa bản copy vừa tạo để kho sạch
            try:
                os.remove(dst_path)
            except:
                pass
            continue

        # file mới
        try:
            docs = loader.load_pdf(dst_path)
        except Exception as e:
            # nếu không đọc được thì bỏ và xóa
            try: os.remove(dst_path)
            except: pass
            continue

        chunks = splitter.split(docs)
        if chunks:
            vdb.add_documents(chunks)
            HASH_DB[file_hash] = {
                "file": os.path.basename(dst_path),
                "added_at": datetime.now().isoformat()
            }
            added_files.append(os.path.basename(dst_path))
            new_docs_total += len(docs)
            new_chunks_total += len(chunks)
        else:
            # chunk rỗng thì bỏ
            try: os.remove(dst_path)
            except: pass

    save_hash_db(HASH_DB)

    msg = f"✅ Added: {len(added_files)} file(s), pages: {new_docs_total}, chunks: {new_chunks_total}\n"
    if added_files:
        msg += "  + " + ", ".join(added_files) + "\n"
    if skipped_files:
        msg += f"⚠️ Skipped (duplicate by hash): {len(skipped_files)}\n"
        msg += "  - " + ", ".join(skipped_files)

    return msg.strip()

def list_pdf_names(data_dir):
    return [os.path.basename(p) for p in sorted(glob.glob(os.path.join(data_dir, "*.pdf")))]


<H4>Top-k kèm similarity score (Chroma)</H4>

In [16]:
def retrieve_with_scores(query: str, k: int = 4):
    """
    Trả về (doc, score). Với Chroma: score thường là distance (thấp hơn = giống hơn)
    """
    try:
        pairs = vdb.db.similarity_search_with_score(query, k=k)
        return pairs
    except Exception as e:
        return []

def format_scored_context(pairs):
    if not pairs:
        return "Không lấy được context/score."
    blocks = []
    for doc, score in pairs:
        src = doc.metadata.get("source_file", "unknown")
        page = doc.metadata.get("page", "?")
        text = (doc.page_content or "").strip()
        text = text[:1200]  # cắt bớt cho UI đỡ dài
        blocks.append(f"[{src} | page {page} | score={score:.4f}]\n{text}")
    return "\n\n---\n\n".join(blocks)

def answer_with_scored_ctx(q: str):
    ans = answer_question(q)
    pairs = retrieve_with_scores(q, k=6)
    ctx = format_scored_context(pairs)
    return ans, ctx


<H4>UI: click PDF để “mở/tải” + upload ingest chống trùng + context có score</H4>

In [ ]:
import gradio as gr

def ui_refresh_choices():
    names = list_pdf_names(DATA_DIR)
    return gr.update(choices=names, value=names[0] if names else None)

def ui_open_file(selected_name):
    if not selected_name:
        return None
    path = os.path.join(DATA_DIR, selected_name)
    return path if os.path.exists(path) else None

def ui_upload_and_ingest(files):
    msg = ingest_uploaded_pdfs_no_dup(files, loader, splitter, vdb, DATA_DIR)

    # refresh chain (để chắc kèo)
    global retriever, rag_chain, ctx_chain
    retriever = vdb.get_retriever(k=4)
    rag_chain = rag.get_chain(retriever)
    ctx_chain = rag.get_context_only(retriever)

    # refresh list & open first file
    names = list_pdf_names(DATA_DIR)
    first = names[0] if names else None
    return msg, ui_refresh_choices(), ui_open_file(first)

METHOD_MD = """
## 🧠 Phương pháp & cách thức hoạt động (Demo RAG)

### A. Ingest (Nạp PDF)
1) Upload PDF → copy vào kho (`data_source/generative_ai`)
2) Tính **SHA256 hash** → nếu đã có hash thì **skip** (chống trùng)
3) Trích xuất text từ PDF (PyPDFLoader)
4) Làm sạch tiếng Việt + chunking (chunk_size=400, overlap=120)
5) Embedding → lưu vào ChromaDB (persist)

### B. Retrieval + Answer (RAG)
1) Câu hỏi → truy vấn Chroma **Top-k similarity**
2) Lấy các chunk liên quan + **score** (để giải thích vì sao nó chọn đoạn đó)
3) Nhét context vào prompt → LLM sinh câu trả lời
4) UI show context + score để chứng minh “có dẫn chứng”
"""

with gr.Blocks(title="RAG - Hệ thống hỏi đáp luật giao thông") as demo:
    gr.Markdown("# 📌 RAG – HỆ THỐNG HỎI ĐÁP LUẬT GIAO THÔNG")

    with gr.Row():
        # LEFT
        with gr.Column(scale=2):
            question = gr.Textbox(label="Câu hỏi", placeholder="Nhập câu hỏi về nội dung trong PDF...", lines=3)
            btn = gr.Button("Gửi", variant="primary")
            answer = gr.Textbox(label="Câu trả lời", lines=14, interactive=False)

            gr.Markdown("## 🔎 Context (Top-k chunks + similarity score)")
            context = gr.Textbox(label="Top-k Context (score)", lines=12, interactive=False)

            btn.click(fn=answer_with_scored_ctx, inputs=question, outputs=[answer, context])

        # RIGHT
        with gr.Column(scale=1):
            with gr.Tabs():
                with gr.Tab("📚 Tài liệu & Upload"):
                    gr.Markdown("## 📚 Danh sách tài liệu đã nạp")
                    pdf_picker = gr.Radio(
                        choices=list_pdf_names(DATA_DIR),
                        label="Chọn tài liệu",
                        value=(list_pdf_names(DATA_DIR)[0] if list_pdf_names(DATA_DIR) else None)
                    )

                    # “Mở/tải” file
                    pdf_file = gr.File(label="Mở/Tải PDF", value=ui_open_file(pdf_picker.value))
                    pdf_picker.change(fn=ui_open_file, inputs=pdf_picker, outputs=pdf_file)

                    gr.Markdown("## ➕ Nạp thêm PDF mới")
                    uploader = gr.File(label="Chọn file PDF", file_types=[".pdf"], file_count="multiple")
                    ingest_btn = gr.Button("Nạp & cập nhật kho", variant="secondary")
                    ingest_status = gr.Textbox(label="Trạng thái", lines=4, interactive=False)

                    ingest_btn.click(fn=ui_upload_and_ingest, inputs=uploader, outputs=[ingest_status, pdf_picker, pdf_file])

                with gr.Tab("🧩 Phương pháp"):
                    gr.Markdown(METHOD_MD)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://cfa5373e1f0f39cba4.gradio.live

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
